# 컬럼 조정으로 f1 score 향상되는지 확인

In [16]:
import pandas as pd
df = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')

In [17]:
df.columns
# df.info()

Index(['청구서번호', 'No.', 'Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1',
       'Part No.2', 'key1', 'key2', '리드타임_음수제거', '청구량', '견적', '견적수량', '견적화폐',
       '견적단가', '발주번호', '발주처', '발주', '발주수량', '발주금액', 'D/T', '미입고 기간', '리드타임',
       '창고입고', '창고입고수량', 'Control No.', '입고창고', '창고출고', '창고출고수량', '출고선박',
       '출고운반선', '선박입고', '선박입고수량', '완료 여부'],
      dtype='object')

# 분류에 필요없다고 판단되는 컬럼 드랍

In [28]:
# X 에 쓰일 컬럼 후보들 key2 는 정답이라서 제거하였음
df1 = df.drop(['청구서번호','No.','key2',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부'], axis=1)

In [29]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      20517 non-null  object
 1   Machinery    20517 non-null  object
 2   Assembly     20517 non-null  object
 3   청구품목         20517 non-null  object
 4   Part No.1    20517 non-null  object
 5   Part No.2    20517 non-null  object
 6   key1         20517 non-null  object
 7   발주처          20517 non-null  object
 8   D/T          20517 non-null  object
 9   Control No.  20517 non-null  object
dtypes: object(10)
memory usage: 1.6+ MB


## 컬럼 중 5개를 뽑는 경우의 수

In [30]:
import itertools

# 데이터프레임의 열 목록
columns = df1.columns.tolist()

# 5개의 열로 가능한 모든 조합 생성
combinations = list(itertools.combinations(columns, 5))

# 조합 개수 출력
print(len(combinations))

252


In [31]:
combinations = [list(combo) for combo in combinations]

In [53]:
# combinations

In [23]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.metrics import accuracy_score, f1_score
y = df["key2"]
le_y = preprocessing.LabelEncoder()
y = le_y.fit_transform(y)

In [32]:
# 학습할 정답 컬럼
y

array([18, 18, 18, ...,  5, 20, 20])

In [33]:
def making_X(combo):
    le_x = preprocessing.LabelEncoder()
    X = df1[combo]
    X = X.apply(le_x.fit_transform)
    return X

In [34]:

def train_model(combo, X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier(objective='multi:softmax', num_class=61)

    # 모델 학습
    model.fit(X_train, y_train)
    # 예측 결과 출력
    y_pred = model.predict(X_test)
    # 리턴 값
    accu_score =  accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accu_score,f1

In [35]:
resultset = {}
max_accu = 0
max_f1=0
for combo in combinations:
    X = making_X(combo)
    a, f1 = train_model(combo, X,y)
    if a > max_accu:
        max_accu = a
        resultset['max_accu'] = max_accu
        resultset['max_accu_combo'] = combo
    if f1 > max_f1:
        max_f1 = f1
        resultset['max_f1'] = max_accu
        resultset['max_f1_combo'] = combo
    print('combo : ', combo)
    print('accuracy_score : ' , a)
    print('f1_score : ' , f1) 
    print('---------------------------')

combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1']
accuracy_score :  0.8294346978557505
f1_score :  0.8276234903237275
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.2']
accuracy_score :  0.7485380116959064
f1_score :  0.7465095928271847
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'key1']
accuracy_score :  0.8299220272904484
f1_score :  0.8280752393210912
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', '발주처']
accuracy_score :  0.7443957115009746
f1_score :  0.7415271069114773
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'D/T']
accuracy_score :  0.7426900584795322
f1_score :  0.7397932046486829
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Control No.']
accuracy_score :  0.7409844054580896
f1_score :  0.7381213795133166
---------------------------
combo :  ['Subject', 'Machinery', 'As

###  5개 조합 중에 정확도와 f1 점수가 가장 높을때 각각의 combination

In [36]:
resultset

{'max_accu': 0.8598927875243665,
 'max_accu_combo': ['Machinery', '청구품목', 'Part No.1', 'Part No.2', '발주처'],
 'max_f1': 0.8598927875243665,
 'max_f1_combo': ['Machinery', '청구품목', 'Part No.1', 'Part No.2', '발주처']}

## 컬럼 중에 7개를 뽑는 경우의 수

In [37]:
import itertools

# 데이터프레임의 열 목록
columns = df1.columns.tolist()

# 5개의 열로 가능한 모든 조합 생성
combinations7 = list(itertools.combinations(columns, 7))

# combinations 내부 객체를 list로 형변환
combinations7 = [list(combo) for combo in combinations7]

# 조합 개수 출력
print(len(combinations7))

120


In [ ]:
def making_X(combo):
    le_x = preprocessing.LabelEncoder()
    X = df1[combo]
    X = X.apply(le_x.fit_transform)
    return X

def train_model(combo, X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier(objective='multi:softmax', num_class=61)

    # 모델 학습
    model.fit(X_train, y_train)
    # 예측 결과 출력
    y_pred = model.predict(X_test)
    # 리턴 값
    accu_score =  accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accu_score,f1

### f1 score 83점 이상만 출력

In [40]:
resultset7 = {}
cnt = 0
for combo in combinations7:
    X = making_X(combo)
    a, f1 = train_model(combo, X,y)
    if f1 < 0.83: # f1이 83% 보다 낮으면 다음 모델
        continue
    cnt += 1
    print('combo : ', combo)
    combo.append(a)
    combo.append(f1)
    resultset7[cnt] = combo
    print('accuracy_score : ' , a)
    print('f1_score : ' , f1) 
    print('---------------------------')

combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', '발주처']
accuracy_score :  0.8360136452241715
f1_score :  0.8346144229925088
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'key1', '발주처']
accuracy_score :  0.8345516569200779
f1_score :  0.8329887095025084
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1', '발주처', 'D/T']
accuracy_score :  0.8340643274853801
f1_score :  0.8318964868286428
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1', '발주처', 'Control No.']
accuracy_score :  0.8355263157894737
f1_score :  0.8332792387516869
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.2', 'key1', '발주처']
accuracy_score :  0.8372319688109162
f1_score :  0.8356804264918806
---------------------------
combo :  ['Subject', 'Machinery', 'Assembly', '청구품목', 'key1', '발주처', 'Control No.']
accuracy_score 

In [47]:
# resultset7

###  7개 조합 중에 f1 점수가  83 이상인 combination, 정확도, f1

In [52]:
for i in resultset7.values():
    if i[8] >0.845:
        print(i)

['Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처', 0.8474658869395711, 0.8460942909188021]
['Machinery', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처', 'D/T', 0.8508771929824561, 0.8493876154057275]
['Machinery', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처', 'Control No.', 0.8520955165692008, 0.8504970115225794]
['Machinery', '청구품목', 'Part No.1', 'Part No.2', '발주처', 'D/T', 'Control No.', 0.8494152046783626, 0.8475743698760492]
['Machinery', '청구품목', 'Part No.2', 'key1', '발주처', 'D/T', 'Control No.', 0.8467348927875243, 0.8452176062465274]
['청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처', 'D/T', 'Control No.', 0.8477095516569201, 0.8458364830078636]
